In [0]:
# Step 1: Download a book from an external source to file system 

%sh
wget -P /tmp https://www.gutenberg.org/cache/epub/1513/pg1513.txt

--2025-03-04 17:54:36--  https://www.gutenberg.org/cache/epub/1513/pg1513.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169541 (166K) [text/plain]
Saving to: ‘/tmp/pg1513.txt.2’

     0K .......... .......... .......... .......... .......... 30%  435K 0s
    50K .......... .......... .......... .......... .......... 60%  628K 0s
   100K .......... .......... .......... .......... .......... 90%  628K 0s
   150K .......... .....                                      100% 28.0M=0.3s

2025-03-04 17:54:37 (602 KB/s) - ‘/tmp/pg1513.txt.2’ saved [169541/169541]

Rserv
RtmpMOJO88
chauffeur-daemon-params
chauffeur-daemon.pid
chauffeur-env.sh
custom-spark.conf
driver-daemon-params
driver-daemon.pid
driver-env.sh
hsperfdata_root
pg1513.txt
pg1513.txt.1
pg1513.txt.2
systemd-private-31b3df35480c472b9aee9c4

In [0]:
book = sc.textFile("file:///tmp/pg1513.txt")

In [0]:
book.take(10)

Out[18]: ['The Project Gutenberg eBook of Romeo and Juliet',
 '    ',
 'This ebook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. You may copy it, give it away or re-use it under the terms',
 'of the Project Gutenberg License included with this ebook or online',
 'at www.gutenberg.org. If you are not located in the United States,',
 'you will have to check the laws of the country where you are located',
 'before using this eBook.',
 '']

In [0]:
words = book.flatMap(lambda x: x.split(" ")).map(lambda x: x.lower())
longWords = words.filter(lambda x: len(x) > 5)
wordPairs = longWords.map(lambda x: (x, 1))
wordCounts = wordPairs.reduceByKey(lambda x,y: x + y).sortBy(lambda x: -x[1])


In [0]:
wordCounts.take(10)

Out[20]: [('romeo.', 188),
 ('juliet.', 132),
 ('capulet.', 108),
 ('nurse.', 104),
 ('project', 84),
 ('mercutio.', 67),
 ('benvolio.', 65),
 ('lawrence.', 58),
 ('gutenberg™', 54),
 ('should', 50)]

In [0]:
%sh curl https://files.grouplens.org/datasets/movielens/ml-latest-small.zip --output /tmp/ml-latest-small.zip
unzip /tmp/ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  955k  100  955k    0     0  1538k      0 --:--:-- --:--:-- --:--:-- 1538k
Archive:  /tmp/ml-latest-small.zip


In [0]:
%sh
wget -P /tmp https://raw.githubusercontent.com/a-nagar/datasets/refs/heads/main/movielens/movies.csv
wget -P /tmp https://raw.githubusercontent.com/a-nagar/datasets/refs/heads/main/movielens/ratings.csv
wget -P /tmp https://raw.githubusercontent.com/a-nagar/datasets/refs/heads/main/movielens/tags.csv

--2025-03-04 18:26:31--  https://raw.githubusercontent.com/a-nagar/datasets/refs/heads/main/movielens/movies.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 494431 (483K) [text/plain]
Saving to: ‘/tmp/movies.csv’

     0K .......... .......... .......... .......... .......... 10% 4.94M 0s
    50K .......... .......... .......... .......... .......... 20% 6.17M 0s
   100K .......... .......... .......... .......... .......... 31% 5.96M 0s
   150K .......... .......... .......... .......... .......... 41% 5.97M 0s
   200K .......... .......... .......... .......... .......... 51% 39.3M 0s
   250K .......... .......... .......... .......... .......... 62% 26.8M 0s
   300K .......... .......... .......... .......... .......... 72% 56.3M 0s
   350K ..........

In [0]:
moviesDF = spark.read.option("header","true").option("inferSchema","true").csv("file:///tmp/movies.csv")
ratingsDF = spark.read.option("header","true").option("inferSchema","true").csv("file:///tmp/ratings.csv")
tagsDF = spark.read.option("header","true").option("inferSchema","true").csv("file:///tmp/tags.csv")

In [0]:
moviesDF.show(10)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
+-------+--------------------+--------------------+
only showing top 10 rows



In [0]:
ratingsDF.show(10)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
+------+-------+------+---------+
only showing top 10 rows



In [0]:
tagsDF.show(10)

+------+-------+-----------------+----------+
|userId|movieId|              tag| timestamp|
+------+-------+-----------------+----------+
|     2|  60756|            funny|1445714994|
|     2|  60756|  Highly quotable|1445714996|
|     2|  60756|     will ferrell|1445714992|
|     2|  89774|     Boxing story|1445715207|
|     2|  89774|              MMA|1445715200|
|     2|  89774|        Tom Hardy|1445715205|
|     2| 106782|            drugs|1445715054|
|     2| 106782|Leonardo DiCaprio|1445715051|
|     2| 106782|  Martin Scorsese|1445715056|
|     7|  48516|     way too long|1169687325|
+------+-------+-----------------+----------+
only showing top 10 rows



In [0]:
# Which movie has the highest count of ratings
from pyspark.sql.functions import count, desc
ratingsDF.groupBy("movieId").count().orderBy(desc("count")).show()

+-------+-----+
|movieId|count|
+-------+-----+
|    356|  329|
|    318|  317|
|    296|  307|
|    593|  279|
|   2571|  278|
|    260|  251|
|    480|  238|
|    110|  237|
|    589|  224|
|    527|  220|
|   2959|  218|
|      1|  215|
|   1196|  211|
|     50|  204|
|   2858|  204|
|     47|  203|
|    780|  202|
|    150|  201|
|   1198|  200|
|   4993|  198|
+-------+-----+
only showing top 20 rows



In [0]:
# Find the avg ratings for each movie
avgRatings = ratingsDF.groupBy("movieId").avg("rating")
avgRatings.show()

+-------+------------------+
|movieId|       avg(rating)|
+-------+------------------+
|   1580| 3.487878787878788|
|   2366|              3.64|
|   3175|              3.58|
|   1088| 3.369047619047619|
|  32460|              4.25|
|  44022| 3.217391304347826|
|  96488|              4.25|
|   1238| 4.055555555555555|
|   1342|               2.5|
|   1591|2.6346153846153846|
|   1645| 3.411764705882353|
|   4519|3.3333333333333335|
|   2142|               2.7|
|    471|              3.55|
|   3997|1.8333333333333333|
|    833|               2.0|
|   3918|3.2777777777777777|
|   7982|              3.25|
|   1959|3.6666666666666665|
|  68135|              3.55|
+-------+------------------+
only showing top 20 rows



In [0]:
# Find the movies with the highest avg rating
ratingsDF.groupBy("movieId").avg("rating").toDF("movieId", "avgRating").orderBy(desc("avgRating")).show()

+-------+---------+
|movieId|avgRating|
+-------+---------+
|  26350|      5.0|
|   3795|      5.0|
|  25887|      5.0|
| 157775|      5.0|
|    633|      5.0|
|  33138|      5.0|
|  67618|      5.0|
|    876|      5.0|
|    496|      5.0|
|  27373|      5.0|
| 113829|      5.0|
|  53578|      5.0|
| 152711|      5.0|
| 118894|      5.0|
|     53|      5.0|
| 160644|      5.0|
|    148|      5.0|
|   8911|      5.0|
| 147300|      5.0|
|  84273|      5.0|
+-------+---------+
only showing top 20 rows



In [0]:

# Now join the movies and ratings tables, and give the names of the top 10 movies with the highest ratings
avgRatings = ratingsDF.groupBy("movieId").avg("rating").toDF("movieId", "avgRating").orderBy(desc("avgRating"))
avgRatings.join(moviesDF, "movieId").orderBy(desc("avgRating")).show()

+-------+---------+--------------------+--------------------+
|movieId|avgRating|               title|              genres|
+-------+---------+--------------------+--------------------+
|  26350|      5.0|Passenger, The (P...|               Drama|
|   3795|      5.0|Five Senses, The ...|               Drama|
|  25887|      5.0|Tales of Manhatta...|        Comedy|Drama|
| 157775|      5.0|Tenchi Muyô! In L...|    Animation|Comedy|
|    633|      5.0|Denise Calls Up (...|              Comedy|
|  33138|      5.0|  Palindromes (2004)|Adventure|Comedy|...|
|  67618|      5.0|Strictly Sexual (...|Comedy|Drama|Romance|
|    876|      5.0|Supercop 2 (Proje...|Action|Comedy|Cri...|
|    496|      5.0|What Happened Was...|Comedy|Drama|Roma...|
|  27373|      5.0|          61* (2001)|               Drama|
| 113829|      5.0|One I Love, The (...|Comedy|Drama|Romance|
|  53578|      5.0|Valet, The (La do...|              Comedy|
| 152711|      5.0|Who Killed Chea V...|         Documentary|
| 118894

In [0]:
# Now join the movies and tags tables, and output the NAMES all movies with the tag 'mathematics'
tagMovies = tagsDF.join(moviesDF, "movieId")
tagMovies.filter("tag like 'math%'").show()


+-------+------+-----------+----------+--------------------+--------------------+
|movieId|userId|        tag| timestamp|               title|              genres|
+-------+------+-----------+----------+--------------------+--------------------+
|   1921|   474|mathematics|1138031797|           Pi (1998)|Drama|Sci-Fi|Thri...|
|   4995|   305|mathematics|1464428783|Beautiful Mind, A...|       Drama|Romance|
|  36527|   474|mathematics|1137201998|        Proof (2005)|               Drama|
+-------+------+-----------+----------+--------------------+--------------------+

